In [1]:
import sys
import os
import os.path

import numpy as np
import skimage.io
import glob

import shutil
#from config import config_vars

from tqdm import tqdm

import matplotlib.pyplot as plt
import skimage.morphology

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder
from skimage.transform import resize
from skimage import measure
from skimage import filters

Using TensorFlow backend.


#### Load images 

In [9]:
filename = sorted(os.listdir('/home/fvitez/nuclei_detection/aitslab_boundary_labels/resized_images/'))
image_names = [os.path.join('/home/fvitez/nuclei_detection/resized_aitslab_png_images/', f) for f in filename]
#image_names.remove('/home/jupyter/aitslab_images/norm_images/.ipynb_checkpoints')

In [11]:
imagebuffer = skimage.io.imread_collection(image_names)
images = imagebuffer.concatenate()

#### Make predictions with Otsus and save  

In [16]:
config_vars = {
    'boundary_size': 2
}

In [19]:
annotation_dir = "/home/fvitez/nuclei_detection/otsus_prediction/annotation/"
labels_dir = "/home/fvitez/nuclei_detection/otsus_prediction/labels/"


for n,i in enumerate(images):
    thresh_val = skimage.filters.threshold_otsu(i) #find threshold using otsus method
    mask = i > thresh_val # create mask 
    object_labels = skimage.measure.label(mask) # convert to skimage label
    annot = skimage.morphology.label(object_labels) # convert to annotation
    
    skimage.io.imsave(annotation_dir + filename[n], annot)
    
    #annot = skimage.morphology.remove_small_objects(annot, min_size=config_vars["min_nucleus_size"]) #unclear if necessary
    boundaries = skimage.segmentation.find_boundaries(annot) #extract boundaries
    
    #plt.imshow(annot)
    #break
    
    for k in range(2, config_vars["boundary_size"], 2):
        boundaries = skimage.morphology.binary_dilation(boundaries)
    
    # prepare buffer for binary label
    label_binary = np.zeros((annot.shape + (3,)))

    # write binary label
    label_binary[(annot == 0) & (boundaries == 0), 0] = 1
    label_binary[(annot != 0) & (boundaries == 0), 1] = 1
    label_binary[boundaries == 1, 2] = 1
    
    #plt.imshow(label_binary)
    #break
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(labels_dir + filename[n], label_binary)

/home/fvitez/.conda/envs/nuclei_detection/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: /home/fvitez/nuclei_detection/otsus_prediction/annotation/MFGTMPcx7_170702000001_G07f02d0.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
Lossy conversion from int64 to uint8. Range [0, 52]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/home/fvitez/.conda/envs/nuclei_detection/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: /home/fvitez/nuclei_detection/otsus_prediction/annotation/MFGTMPcx7_170702090001_B22f15d0.png is a low contrast image
  # This is added back by InteractiveShellApp.init_path()
Lossy conversion from int64 to uint8. Range [0, 75]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint